# Lattice Boltzmann models in 1d

## Diffusion equation

The diffusion equation in one spatial dimension describes propagation of a scalar quantity in time. 
It is widely used to as a macrocopic model of molecular diffusion in the continous limit or temperature propagation in a medium. We will consider the time evolution of a the scalar field $T=T(x,t)$. Then the diffusion equation reads:

$$ \frac{\partial T}{\partial t} = D \frac{\partial^2 T}{\partial x^2} $$

It is parabolic linear partial differential equation which can be solved analytically in many cases. The solution is unique if appropriate boundary conditions and the initial  are given. For example if one specifies the value of the scalar $T$ on the end of interval $(a,b)$ e.g. $T(a,t)=T_1$ and $T(b,t)=T_2$ and its initial values inside the interval $T(x,t=0)=f(x)$, the the solution is unique. 

Here we will solve the diffusion equation with the Latiice Boltzmann Method. It will allow to analyze the properties of the LBM 

## Model D1Q2

We will used LBM method with the D1Q2 grid.
 
 - $f^1$ is the number of particles at $c=1$ and $f^2$ with $c=-1$
 - the equilibrium function does not depend on speed and is equal to $$f_i^{eq}(T) = w_i T$$ with $w_i=(1/2,1/2)$ weights
 - we consider the 1d grid of $x_k$ points for which $f^i(x_k)$ distributes data in each point
 - collision operator: $$-\omega (f-f_{eq})$$
  - $\omega$ relaxation constant links the microscopic and macroscopic description. It can be shown that for the mesh model to approximate the diffusion equation, the following value must be taken:
 $$\omega=\frac{1}{\frac{Dyf}{c^2_s}+0.5}$$
  - $c_s$ has interpretation of the speed of sound on the network and in the case of D1Q2 takes the value 1
 - boundary conditions:
  - consider the reflection at the right end: $f^i(x_{-1})=f^i(x_{-2})$ for $i\in\{1,2\}$
  - and the set value on the right: $f^2(x_0) + f^1(x_0) = T_0$

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
lx = 100

Tend = 100
w  =  np.array([1/2.,1/2.])
cs2 = 1.0
c = np.array([1,-1])

def f_eq(T,w):
    return w[:,np.newaxis]*T

Dyf = 9.5
omega = 1/(Dyf + 0.5)
Tw = 1.0

T_init = 0*np.ones(lx)
f = f_eq(T_init,w)

x = np.linspace(0,lx-1,lx)

T_lst = [T_init]
for iteration in range(Tend):

    # collision 
    f[0,0] = Tw - f[1,0]
    
    # symetryczne odbicie (lub bounce-back ponizej)
    #for i,k in enumerate(c):
    #    f[i,-1] = f[i,-2]    
   
    T = np.sum(f,axis=0)
    
    # equilibrium function 
    fOut = f - omega * (f-f_eq(T,w))
    #bounce back
    fOut[0,-1], fOut[1,-1] = f[1,-1], f[0,-1]  

    # propagation 
    for i,k in enumerate(c):
        f[i,:] = np.roll(fOut[i,:],k,axis=0)
    
    f[0,0] = np.nan
    f[1,-1] = np.nan
    
    if iteration%1==0:
        T_lst.append( T )
        
print("omega=",omega,"Dyf=",Dyf)

*Note* - the streaming is implemented using `np.roll` function. It would produce an artifact at the ends as  the distributions are wrapped priodically (we do not have periodic boundary conditions). The actual values ​​of the $T$ field on the edge are given by the boundary condition. On the other hand the values of distributions:

$$f_0(x=0), f_1(x=-1)$$

are unknown.

## Analysis of solutions

As a result of the above code, we have received a time evolution of $f_1$ and $f_2$ for $t\in(0,100)$.

In the table `T_lst` we have a record of all steps. We can draw the last of them on the chart:

In [ ]:
T = T_lst[-1]     
plt.plot(x,T,'o-')
plt.ylim(0,Tw)
plt.xlim(0,None)
plt.show()

### Distributions

What do the $f_1(x)$ and $f_2(x)$ distributions look like in 100 steps?

Note that in balance we have:

$$f_1 = \frac{1}{2}T \\ f_2 = \frac{1}{2} T$$

From this it follows that the difference between distributors is proportional to how far the system is from equilibrium. For $\omega=0.1$, the state of the system is clearly far from balance.

*Investigate how it will look for larger $\omega$ (change Dyf)*

In [ ]:
plt.figure()
plt.plot(x,f[0,:],'r')
plt.plot(x,f[1,:],'b')
plt.show()

### Numerical reference solution of the diffusion equation

In [ ]:
dt = 1.0/20

dx = 1.0
nt = int(Tend/dt)
u0 = np.zeros(lx)     

#x = np.linspace(0,2,nx)
print(nt*dt)
u = u0.copy()
for n in range(nt):  
    u[1:-1] = u[1:-1] +  Dyf*dt/dx**2*np.diff(u,2) #(u[2:]-2*u[1:-1]+u[:-2])
    u[0] = Tw    
    u[-1] = u[-2]
    

In [ ]:
plt.figure()
plt.plot(x,u,label='FDM@dt=%0.4f'%dt)
plt.plot(x,T,label='LBM D1Q2')
plt.ylim(0,Tw)
plt.xlim(0,None)
plt.legend()
plt.show()

### Analysis of results

For the parameter `Dyf = 9.5`

Comparing the exact numerical solution with the solution obtained by the LBM method, we can see that there were discrepancies.
It should be noted that

 - the LBM model made 100 time steps on a 100-node grid. This means that with such parameters, the network model was practically unable to "penetrate" $x=100$.
 - distribution value analysis shows that with these parameters the model works at the low relaxation constant $\omega$ regime and therefore at every point the system is relatively far from equilibrium. The accuracy of BGK approximation assumes that we are close to balance.
 - The numerical model becomes stable for a time step by an order of magnitude smaller than the step of the LBM model (i.e. $\Delta t = 1$)

## Model D1Q3

We will perform calculations by adding a zero vector to the set of velocity vectors.
It should be changed:

 - $c_s^2 = \frac{1}{3}$ sound speed
 - determine new weights in equilibrium function
 - adjust the boundary condition to the new $f_i$ set - if we use bounce-back then only the Dirichlet condition $T(x=0)=1$.

In [ ]:
#D1Q3 ()
import numpy as np
import matplotlib.pyplot as plt

lx = 100

w  =  np.array([4/6., 1/6.,1/6.])
cs2 = 1/3.0
c = np.array([0,1,-1])


def f_eq(T,w):
    return w[:,np.newaxis]*T


omega = 1/(3*Dyf+0.5)
Tw = 1

T_init = 0*np.ones(lx)
f = f_eq(T_init,w)

T_lst = [T_init]
for iteration in range(Tend):

    f[1,0] = 1/3.0*Tw - f[2,0]
    f[0,0] = Tw*2/3.0
   
    #for i,k in enumerate(c):
    #    f[i,-1] = f[i,-2]
        
    T = np.sum(f,axis=0)
    fOut = f - omega * (f-f_eq(T,w))
    
    # bounce back
    fOut[1,-1],fOut[2,-1] =  f[2,-1],f[1,-1]  


    for i,k in enumerate(c):
        f[i,:] = np.roll(fOut[i,:],k,axis=0)
    
    if iteration%1==0:
        T_lst.append( T )
        
 
print(omega,Dyf)

In [ ]:
plt.figure()
plt.plot(x,f[0,:],'r')
plt.plot(x,f[1,:],'g')
plt.plot(x,f[2,:],'b')
plt.plot(x,np.sum(f,axis=0),'k-.')
plt.show()

In [ ]:
def res_D1d(u,u0,Dyf=Dyf):
    resL = u[1:-1] -u0[1:-1]
    resR = Dyf*np.diff(u0,2)
    res = resL-resR
    return res

### Solution of the D1Q3 model

*Note* - the analysis of the solution shows that the boundary condition is not correctly set (although the residue for $x>0$ is small). It should be noted that for $D=9.5$ the model is far from equilibrium and in $x=0$ we set the equilibrium condition. This is the reason for the discrepancy. This can be improved, e.g. by bringing the model closer to balance by scaling what is done below.

In [ ]:
plt.figure(figsize=(8,6))
plt.plot(x,u,label='FDM@dt=%0.4f'%dt)
plt.plot(np.linspace(0,100-1,lx),T_lst[-1],'r-',label='LBM D1Q3@n=%d'%lx)
res = res_D1d(T_lst[-1],T_lst[-2])
plt.plot(np.linspace(0,100-1,lx)[1:-1],100*res,label='residuum x 100')
plt.ylim(0,Tw)
plt.xlim(0,None)
plt.legend()
plt.show()

## Scaling

Scaling $t=t^* \tau$; $x=x^* l_0$ leads to the equation:
$$ \frac{\partial T}{\partial t^{*}} = D \frac{\tau}{l_0^2} \frac{\partial^2 T}{\partial x^{*2}} $$

So in scaled units we have:

$$D_{lu} = D \frac{\tau}{l_0^2} $$


From this it follows that we can lower 2x the diffusion constant in $D_{lu}$ network units (and thus incur the relaxation parameter) in two ways:

 - reducing the time step twice
 - reducing the number of nodes by $\sqrt{2}$

In [ ]:
#D1Q3 ()
import numpy as np
import matplotlib.pyplot as plt

def solve_diff(a=1,b=1,Dyf = 9.5,time_evo=False):
  

    lx = int(100/np.sqrt(a))
    x = np.linspace(0,99-1,lx)
    
    w  =  np.array([4/6., 1/6.,1/6.])
    cs2 = 1/3.0
    c = np.array([0,1,-1])


    def f_eq(T,w):
        return w[:,np.newaxis]*T


    omega = 1/(3*Dyf/(a*b)+0.5)
    Tw = 1

    T_init = 0*np.ones(lx)
    f = f_eq(T_init,w)

    #f0_eq = f_eq(np.array([Tw]),w)

    T_lst = [T_init]
    for iteration in range(int(b*Tend)):

        f[1,0] = 1/3.0*Tw - f[2,0]
        f[0,0] = Tw*2/3.0

        #for i,k in enumerate(c):
        #    f[i,-1] = f[i,-2]


        T = np.sum(f,axis=0)


        fOut = f - omega * (f-f_eq(T,w))

        # bounce back
        fOut[1,-1],fOut[2,-1] =  f[2,-1],f[1,-1]  


        for i,k in enumerate(c):
            f[i,:] = np.roll(fOut[i,:],k,axis=0)

        T_lst.append(T)
    
    if time_evo:
        return x,T_lst
    else:
        print("omega=",omega,"steps:",int(b*Tend),'size:',lx)

        return x,T
 


In [ ]:
plt.figure()
plt.plot(*solve_diff(a=16,b=1),'ro-')
plt.plot(x,u,label='FDM@dt=%0.4f'%dt)
plt.legend()
plt.show()

### Scaling the model

e.g:

 - $a=4$ means reduce the number of nodes by 2 times
 - $b=2$ means reduce the time step 2 times

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
from ipywidgets import  interact,Layout
from ipywidgets.widgets import  FloatSlider
style = Layout(width='70%')
f,ax = plt.subplots(figsize=(8,5))
ax.plot(np.linspace(0,99,100),u,label='FDM@dt=%0.4f'%dt)

lbm_plt = ax.plot([0],[0],'ro-')[0]
f.canvas.draw()
plt.show()
@interact(a=FloatSlider(min=1e-2,max=100,step=0.001,value=1.,layout=style),\
         b=FloatSlider(min=1e-2,max=10,step=0.001,value=1.,layout=style))
def _(a,b):
    lbm_plt.set_data(*solve_diff(a=a,b=b))


### Time propagation

In [ ]:
%matplotlib notebook

from ipywidgets.widgets import IntSlider,Layout
style = Layout(width='70%')

x,Tlst = solve_diff(a=4,b=1,time_evo=True)
f,ax = plt.subplots(figsize=(8,5))

ax.plot(np.linspace(0,99,100),u,label='FDM@dt=%0.4f'%dt)

lbm_t_plt = ax.plot([0],[0],'ro-')[0]
f.canvas.draw()
plt.show()
@interact(ith=IntSlider(min=0,max=len(Tlst)-1,layout=style))
def _(ith):
    lbm_t_plt.set_data(x,Tlst[ith])


\newpage